In [1]:
import sys
import json
import torch
sys.path.append('/home/kshitij/legaleval-subtask-a')

from collections import Counter
from models.CNN_BiLSTM import CNN_BiLSTM
from utils import sent2embeddings, label_encode
import torch.nn as nn

# Data

In [2]:
with open("../data/train.json") as json_file:
    data = json.load(json_file)

In [3]:
print(f"Number of documents : {len(data)}")

Number of documents : 247


#### Looping through each document

In [4]:
total_sentences = 0
group = []
for index,doc in enumerate(data):
    print(f"Document number : {index}")
    print(f"Number of sentences : {len(doc['annotations'][0]['result'])}")
    total_sentences += len(doc['annotations'][0]['result'])
    group.append(doc['meta']['group'])
    
print(f"Avg sentences = {total_sentences/len(data):.2f}")

Document number : 0
Number of sentences : 91
Document number : 1
Number of sentences : 72
Document number : 2
Number of sentences : 200
Document number : 3
Number of sentences : 119
Document number : 4
Number of sentences : 184
Document number : 5
Number of sentences : 211
Document number : 6
Number of sentences : 140
Document number : 7
Number of sentences : 87
Document number : 8
Number of sentences : 228
Document number : 9
Number of sentences : 99
Document number : 10
Number of sentences : 62
Document number : 11
Number of sentences : 213
Document number : 12
Number of sentences : 111
Document number : 13
Number of sentences : 199
Document number : 14
Number of sentences : 188
Document number : 15
Number of sentences : 271
Document number : 16
Number of sentences : 43
Document number : 17
Number of sentences : 82
Document number : 18
Number of sentences : 171
Document number : 19
Number of sentences : 149
Document number : 20
Number of sentences : 95
Document number : 21
Number of 

In [5]:
print(f"Number of law groups : \n{set(group)}")

Number of law groups : 
{'Tax', 'Criminal'}


#### Looping through each document + sentence

In [6]:
labels = []
for index,doc in enumerate(data):
    char_count = 0
    group = doc['meta']['group']
    print(f"Document number : {index:<6} Law Type : {group}")
    # print(f"Number of sentences : {len(doc['annotations'][0]['result'])}")
    if doc['annotations'][0]['result'] != []:
        for sentence_data in doc['annotations'][0]['result']:
            char_count += len(sentence_data['value']['text'])
            labels.append(sentence_data['value']['labels'][0])
        print(f"Avg number of chars : {char_count/len(doc['annotations'][0]['result']):.2f}")
    else:
        print(f"{'Document is empty':-^60}")

Document number : 0      Law Type : Criminal
Avg number of chars : 180.81
Document number : 1      Law Type : Tax
Avg number of chars : 153.39
Document number : 2      Law Type : Criminal
Avg number of chars : 193.93
Document number : 3      Law Type : Tax
Avg number of chars : 222.87
Document number : 4      Law Type : Tax
Avg number of chars : 183.04
Document number : 5      Law Type : Criminal
Avg number of chars : 143.89
Document number : 6      Law Type : Tax
Avg number of chars : 202.29
Document number : 7      Law Type : Criminal
Avg number of chars : 118.03
Document number : 8      Law Type : Criminal
Avg number of chars : 140.69
Document number : 9      Law Type : Criminal
Avg number of chars : 181.81
Document number : 10     Law Type : Criminal
Avg number of chars : 153.18
Document number : 11     Law Type : Tax
Avg number of chars : 177.47
Document number : 12     Law Type : Tax
Avg number of chars : 156.68
Document number : 13     Law Type : Criminal
Avg number of chars : 1


Document number : 160    Law Type : Criminal
Avg number of chars : 132.71
Document number : 161    Law Type : Tax
Avg number of chars : 138.84
Document number : 162    Law Type : Tax
Avg number of chars : 163.06
Document number : 163    Law Type : Criminal
Avg number of chars : 185.28
Document number : 164    Law Type : Criminal
Avg number of chars : 163.82
Document number : 165    Law Type : Tax
Avg number of chars : 192.76
Document number : 166    Law Type : Tax
Avg number of chars : 199.42
Document number : 167    Law Type : Tax
Avg number of chars : 177.50
Document number : 168    Law Type : Criminal
Avg number of chars : 144.85
Document number : 169    Law Type : Tax
Avg number of chars : 204.69
Document number : 170    Law Type : Criminal
Avg number of chars : 122.97
Document number : 171    Law Type : Criminal
Avg number of chars : 146.98
Document number : 172    Law Type : Criminal
Avg number of chars : 189.48
Document number : 173    Law Type : Criminal
Avg number of chars : 

In [7]:
print(set(labels))

{'NONE', 'STA', 'RLC', 'ARG_RESPONDENT', 'ANALYSIS', 'RPC', 'PRE_NOT_RELIED', 'PREAMBLE', 'PRE_RELIED', 'ARG_PETITIONER', 'RATIO', 'ISSUE', 'FAC'}


In [8]:
Counter(labels)

Counter({'PREAMBLE': 4167,
         'NONE': 1423,
         'FAC': 5744,
         'ARG_RESPONDENT': 698,
         'RLC': 752,
         'ARG_PETITIONER': 1315,
         'ANALYSIS': 10695,
         'PRE_RELIED': 1431,
         'RATIO': 674,
         'RPC': 1081,
         'ISSUE': 367,
         'STA': 481,
         'PRE_NOT_RELIED': 158})

# Training through 1 document

In [9]:
label_encoder = label_encode(list(set(labels)))

In [14]:
model = CNN_BiLSTM()
model_optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
print(model)
print(model_optimizer)

CNN_BiLSTM(
  (word_conv): Sequential(
    (0): Conv2d(1, 1, kernel_size=(5, 1), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
  )
  (sent_conv): Conv2d(3, 1, kernel_size=(1, 1), stride=(1, 1))
  (bilstm): LSTM(768, 256, num_layers=2, batch_first=True, bidirectional=True)
  (dense): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=128, out_features=13, bias=True)
    (3): Softmax(dim=1)
  )
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)


In [15]:
train_document = data[0]['annotations'][0]['result']
len(train_document)

91

In [16]:
def train(model : object, model_optimizer : object,
          inp : torch.TensorType, target : torch.TensorType) -> float:
    """
    Train the decoder model for a single step using the given input and target sequences.

    Args:
        decoder (object): The decoder model to be trained.
        decoder_optimizer (object): The optimizer for updating the decoder's parameters.
        inp (torch.TensorType): The input sequence tensor.
        target (torch.TensorType): The target sequence tensor.

    Returns:
        float: The normalized loss for the current training step, averaged over the sequence length.
    """
    hidden, cell = model.init_hidden()
    model.zero_grad()
    criterion = nn.CrossEntropyLoss()

    output, (hidden, cell) = model(inp, hidden, cell)
    # print(output.size())
    # print(target.size())
    loss = criterion(output, target)

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
    model_optimizer.step()

    return loss.item() 

In [17]:
max_sent_tok_len = 80
all_losses = []

for index, entry in enumerate(train_document):
    sent_triplet = []
    sent_tensor = torch.Tensor()
    if index == 0: # for start of document duplicate 1st sentence
        sent_triplet.append(train_document[index]['value']['text'])
        sent_triplet.append(train_document[index]['value']['text'])
        sent_triplet.append(train_document[index+1]['value']['text'])
    elif index == len(train_document) - 1: # for end of document duplicate last sentence
        sent_triplet.append(train_document[index-1]['value']['text'])
        sent_triplet.append(train_document[index]['value']['text'])
        sent_triplet.append(train_document[index]['value']['text'])
    else:
        sent_triplet.append(train_document[index-1]['value']['text'])
        sent_triplet.append(train_document[index]['value']['text'])
        sent_triplet.append(train_document[index+1]['value']['text'])
        
    for sent in sent_triplet:
        sent_emb = sent2embeddings(sent,MAX_LEN = max_sent_tok_len) 
        sent_tensor = torch.cat((sent_tensor,sent_emb),dim=0)
    
    # print(f"input size : {sent_tensor.size()}")
    target_encoded = torch.from_numpy(label_encoder.transform(entry['value']['labels'])).float()
    # print(f"target size : {target_encoded.size()}")
    # training tensor --> sent_tensor
    # training target --> target_encoded
    
    loss = train(model,model_optimizer,sent_tensor,target_encoded)
    print(loss)
    all_losses.append(loss)

2.561986207962036
2.5300374031066895
2.514539957046509
2.5127615928649902
2.5779592990875244
2.5772600173950195
2.579756259918213
2.5691683292388916
2.5504183769226074
2.552877187728882
2.5322506427764893
2.5722768306732178
2.5926661491394043
2.5883586406707764
2.5596189498901367
2.464824914932251
2.3301966190338135
2.348546266555786
2.22310471534729
2.2851197719573975
2.6240110397338867
2.6310296058654785
2.648003578186035
2.6471028327941895
2.0850958824157715
2.6515121459960938
2.6570653915405273
2.663188934326172
2.656874656677246
2.665992259979248
2.6744253635406494
2.6699719429016113
2.645907163619995
2.6633667945861816
2.6677780151367188
2.671833038330078
2.60341215133667
2.590376615524292
2.5922722816467285
2.594261646270752
2.5348148345947266
2.5120153427124023
2.4590938091278076
2.4181337356567383
2.3873751163482666
2.3286936283111572
2.2155628204345703
2.0890986919403076
2.0147135257720947
2.6570751667022705
2.6585569381713867
2.670090675354004
2.6746301651000977
2.6763095855

KeyboardInterrupt: 

In [18]:
len(all_losses)

77